In [42]:
!pip install pinecone-client openai pypdf langchain langchain-community langchain-openai langchain-pinecone -qU pinecone-notebooks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.1/456.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.


In [46]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
import pinecone
import os
import getpass
import time

In [44]:
pinecone_api = "api_key"

In [48]:
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

# pc = Pinecone(api_key=pinecone_api_key)

In [49]:
from pinecone import Pinecone, ServerlessSpec
import time

pc = Pinecone(api_key=pinecone_api)

In [50]:
spec = ServerlessSpec(cloud="aws", region="us-east-1")

In [51]:
index_name = "rag-test"

In [53]:
pdf_loader = PyPDFLoader("3550-1chapter04.pdf")
documents = pdf_loader.load()

In [54]:
# Chunk the document into smaller segments
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [55]:
from langchain_pinecone import PineconeEmbeddings
import os

model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=pinecone_api)

In [56]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embeddings.dimension,
        metric="cosine",
        spec=spec
    )
    # Wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

In [57]:
# See that it is empty
print("Index before upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")


Index before upsert:
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}




In [58]:
vector_store = PineconeVectorStore(index = pc.Index(index_name), embedding = embeddings)

In [40]:
namespace = "testdoc"

In [59]:
from langchain_core.documents import Document
from uuid import uuid4

In [64]:
vector_store.add_documents(documents=chunks, namespace = namespace)

['29d7b7d3-f00b-4aae-b7bf-a2001b8780d1',
 'c67406a4-920f-48d2-8bd7-b7a474450972',
 'bb5acccc-a21a-43d4-851f-05aecd7a3948',
 '98cab9ef-3b1d-4690-aa4a-957c21fe4096',
 'a7a0d3a3-6613-426f-9958-a77c884dce0b',
 'd12f6aad-1639-4d0f-a979-699adc0549dd',
 'd3ac4fa1-dfa1-4fa2-982e-0b09205b37a1',
 'e82af344-f51a-4285-bff1-3ed3263bc64b',
 'bd95c998-4ba0-42c6-ae9f-a05ad5ac21aa',
 '6b7c2376-6558-4351-8968-3f4e2c46ef52',
 '305682a7-c275-4ab0-a4da-fac3c94e542a',
 '5e2a9ec8-b017-419d-9a31-baa67199532e',
 '3641aa1b-fa6b-4923-8367-7100d279b01e',
 '80561412-07fc-4f45-99c1-9274fe7ee20d',
 '5aaf0673-ef09-42c1-b04e-37ee5e7d1c80',
 '37a235a5-7d71-42e1-b767-8322ab92827e',
 '7f1110d6-ccef-4aee-aea0-ba660a1eb642',
 '0850cd12-3aba-485d-9709-52ba4e005a94',
 '9ef23ec5-69f0-4118-93a1-8bcb6eaaa7ab',
 '3aed6611-aa22-4ba0-ae39-8008301ed78d',
 '4ea482c9-2cd4-44ff-a154-c96bbc39dcee',
 '8128a2a8-f882-4572-a8d7-5c9245d398c4',
 'cdf9f4d6-ce70-4836-8d56-ede87762b99a',
 'ba38bc67-f95e-4c8f-a41f-6c0b0b48160f',
 '2da63e21-9be4-

In [65]:
print("Index after upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")
time.sleep(2)


Index after upsert:
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 248}},
 'total_vector_count': 248}


